# Machine Learning: Random Forest Regressor predictions
* The RFR seems to be the best prediction model: 

In [ ]:
##### LIBRARIES IN USE #########
import pandas as pd
import numpy as np

##### DataViz ##########
import matplotlib.pyplot as plt
import seaborn as sns

####### Machine Learning ##########
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import GridSearchCV
from prophet import Prophet


## 1. Preparing the data for the RFR

In [4]:
rfr_pm25meteo = pd.read_csv(r"C:\Users\sophi\FrMarques\LyonData WCS new\P3 wildAir\p3_WildAir\Open_Meteo_com\OpenMeteo_data\CSV\CSV_meteopollu_final\meteopolluwind2124_final.csv")

In [ ]:
rfr_pm25meteo = rfr_pm25meteo.drop(columns="Unnamed: 0")
############## Setting "Date" as the index of the dataset if it is not:
rfr_pm25meteo.set_index('date_id', inplace=True)

# Setting the index ("Date") as "datetime" dtype:
rfr_pm25meteo.index = pd.to_datetime(rfr_pm25meteo.index)

rfr_pm25meteo = rfr_pm25meteo[['temp_c', 'humidity_%', 'rain_mm', 'snowfall_cm',
       'atmopressure_hpa', 'cloudcover_%', 'u10',
       'v10', 'PM2.5']]

In [25]:
rfr_pm25meteo.head(3)

,date_id,temp_c,humidity_%,rain_mm,snowfall_cm,atmopressure_hpa,cloudcover_%,u10,v10,PM2.5,PM2.5_t-1,PM2.5_t-2,PM2.5_t-3
0,2021-01-01,2.599667,85.773796,0.075000,0.002917,988.10660,95.291664,1.368064,-3.283859,24.2,NaN,NaN,NaN
1,2021-01-02,1.618417,79.047424,0.008333,0.023333,989.22394,95.791664,2.309296,-5.272886,16.4,24.2,NaN,NaN
2,2021-01-03,1.016333,78.578060,0.000000,0.017500,991.02985,86.666664,2.735477,-1.686846,55.9,16.4,24.2,NaN


## 2. Random forest Regressor (RFR)

In [12]:
rfr_pm25meteo = rfr_pm25meteo.reset_index()

### 2.1 Setting the RFR with a window slide of 3 days

In [18]:
#### Setting a sliding window of 3 days:
window_size = 3

for i in range(1, window_size + 1):             # loop starting by 1 to create the columns for each window (+1 sets the limit not included of 4) = 1 to 3;
    rfr_pm25meteo[f"PM2.5_t-{i}"] = rfr_pm25meteo["PM2.5"].shift(i)   # the t- window slide dynamic (f) creation where the "i" changes in each loop;

df_sliding = rfr_pm25meteo.dropna().copy()                 # Removing the first lines with "NaN" created by  the shift;

X_sliding = df_sliding.drop(columns=["date_id", "PM2.5"])    # Defining the X with the useful features;
y_sliding = df_sliding["PM2.5"]                              # Defining the target;

split_index = int(len(X_sliding) * 0.8)                      # Defining the teams "Train"/"Test" in a proportion 80/20, keeping
X_train_s, X_test_s = X_sliding.iloc[:split_index], X_sliding.iloc[split_index:]    #... the data cronology, which will
y_train_s, y_test_s = y_sliding.iloc[:split_index], y_sliding.iloc[split_index:]    #... be decisive here;

# X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(        # A more traditional way for split_train_test but;
#     X_sliding, y_sliding, test_size=0.2, shuffle=False, random_state=42   #... imposing no shuffle of data to keep vjronology;
#     )


X_train_s.shape, X_test_s.shape, y_train_s.shape, y_test_s.shape        # checking the size of each of the 4 teams created;


((1166, 11), (292, 11), (1166,), (292,))

### 2.2 Grid Search Cross Validation 
* Tunning the model to get the best params:

In [ ]:
param_grid = {                  # Setting different parameters to retest the RFR 2 times; 
    "n_estimators": [50, 100],  # with 50 trees and 100 trees;
    "max_depth": [10, 20],      # with a depth of 10 and 20 splits;
    "min_samples_split": [2, 5], # Minimum samples before ginfgoing for a new node;
    "min_samples_leaf": [2, 5, 10]  # Minimum samples in a leaf, the last possible node or the external node (1 risks overfitting);
}

rfr_gs = RandomForestRegressor(random_state=42) # creating the base model

grid_search = GridSearchCV(                     # setting the GridSearch to make 3 cross validations;
    estimator=rfr_gs, param_grid=param_grid, 
    cv=3, n_jobs=-1, scoring="neg_mean_absolute_error"
)

grid_search.fit(X_train_s, y_train_s)       # applying the GS;

best_params = grid_search.best_params_      # saving the best hyparameters;

print("Best hyperparameters:", best_params)

rf_best = RandomForestRegressor(**best_params, random_state=42) # Training the best RFR with the optimized params;
rf_best.fit(X_train_s, y_train_s)

y_pred_s = rf_best.predict(X_test_s)            # Making predictions with the "test" teams;

mae_s = mean_absolute_error(y_test_s, y_pred_s) # Evaluating the optimized model;
mse_s = mean_squared_error(y_test_s, y_pred_s)
rmse_s = np.sqrt(mse_s)
r2_s = r2_score(y_test_s, y_pred_s)

print(f"MAE: {mae_s:.2f}")      # Mean Absolute Error (= (Real1 - Pred1) + (Real2 - Pred2) +.../n) limited to 2 decimal cases;
print(f"MSE: {mse_s:.2f}")      # Mean Square Error (= (Real1 - Pred1)2 + (Real2 + Pred2)2 +.../n);
print(f"RMSE: {rmse_s:.2f}")    # Root Mean Square Error(= √mse);
print(f"R²: {r2_s:.2f}")        # Square R (Correlation coefficent square = 0-1 = 0-100%);

C:\Users\sophi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best hyperparameters: {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 100}
MAE: 5.82
MSE: 90.13
RMSE: 9.49
R²: 0.32


### 2.3 RFR Optimized
* with the best hyperparameters proposed by the GridsearchCV:

In [28]:
rfr_optimized = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=10,  # Testing with 10 instead of the proposed 5;
    random_state=42       # saving the samples
)

rfr_optimized.fit(X_train_s, y_train_s)

y_pred_test = rfr_optimized.predict(X_test_s)

mae_test = mean_absolute_error(y_test_s, y_pred_test)
mse_test = mean_squared_error(y_test_s, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test_s, y_pred_test)

print(f"MAE: {mae_test:.2f}")
print(f"MSE: {mse_test:.2f}")
print(f"RMSE: {rmse_test:.2f}")
print(f"R²: {r2_test:.2f}")


MAE: 5.83
MSE: 89.23
RMSE: 9.45
R²: 0.33


## 2.3 Prophet
* Preparing for a fusion of the two models (RFR + P)

In [34]:
df_rfr2prophet = df_sliding.copy()      # the df fro prophet;

df_rfr2prophet = df_rfr2prophet.rename(columns={"date_id": "ds", "PM2.5": "y"}) # getting the column labels needed by Prophet;
df_rfr2prophet["ds"] = pd.to_datetime(df_rfr2prophet["ds"])

df_prophet = df_rfr2prophet[["ds", "y"]].copy()     # isolating the columns needed by prophet;

model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
model_prophet.fit(df_prophet)       # Creating and training the Prophet;

future_p = model_prophet.make_future_dataframe(periods=0)  # Predicting only until the end of data period;
forecast_p = model_prophet.predict(future_p)

df_sliding = df_sliding.merge(forecast_p[["ds", "yhat"]], left_on="date_id", right_on="ds", how="left")
df_sliding = df_sliding.drop(columns=["ds"])  # Adding the Prophet results to the df and removing "ds";

X_fusion = df_sliding.drop(columns=["date_id", "PM2.5"])  # New Train and Test teams
y_fusion = df_sliding["PM2.5"]

split_index = int(len(X_fusion) * 0.8)  # Spliting train and test teams by 80-20%
X_train_f, X_test_f = X_fusion.iloc[:split_index], X_fusion.iloc[split_index:]
y_train_f, y_test_f = y_fusion.iloc[:split_index], y_fusion.iloc[split_index:]

# 🟢 9️⃣ **Treinar o RFR com `yhat` incluído**
rf_fusion = RandomForestRegressor(
    n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=10, random_state=42
)
rf_fusion.fit(X_train_f, y_train_f)     # Training the fusioned RFR with the Prophet "yhat" included; 

y_pred_fusion = rf_fusion.predict(X_test_f)     # Predicting with the fusioned df;

mae_fusion = mean_absolute_error(y_test_f, y_pred_fusion)   # Measuring the efficency; 
mse_fusion = mean_squared_error(y_test_f, y_pred_fusion)
rmse_fusion = np.sqrt(mse_fusion)
r2_fusion = r2_score(y_test_f, y_pred_fusion)

print(f"MAE: {mae_fusion:.2f}")
print(f"MSE: {mse_fusion:.2f}")
print(f"RMSE: {rmse_fusion:.2f}")
print(f"R²: {r2_fusion:.2f}")


17:34:16 - cmdstanpy - INFO - Chain [1] start processing
17:34:17 - cmdstanpy - INFO - Chain [1] done processing


MAE: 5.59
MSE: 84.12
RMSE: 9.17
R²: 0.37


#### Results from the hybrid model:
* The Mean Absolute Error (mae) got the best result with the hybrid RFR/Prophet model: 5.59 from 5.83 (RFR optimized) and 7.68 (Prophet);
* The Mean Square Error also improved: 84.12 from 89.23 and 130.62;
* The RMSE fell to 9.17 from 9.45 and 11.43;
* The R2 reached 37% from 33% and 22%;
